In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Load the data with explicit encoding handling - try multiple encodings
try:
    # Try different encodings
    encodings = ['latin1', 'ISO-8859-1', 'cp1252', 'utf-8-sig']

    for encoding in encodings:
        try:
            print(f"Trying to read file with {encoding} encoding...")
            df = pd.read_csv("/content/Sample - Superstore.csv", encoding=encoding)
            print(f"Successfully read the file with {encoding} encoding.")
            break
        except UnicodeDecodeError:
            print(f"Failed with {encoding} encoding.")
            if encoding == encodings[-1]:
                raise Exception("Could not read the file with any of the attempted encodings.")
            else:
                continue
except Exception as e:
    # If file path is incorrect or file doesn't exist
    print(f"Error accessing the file: {str(e)}")
    print("Attempting to create a sample dataframe based on the example provided...")

    # Create a sample dataframe based on the example
    data = {
        'Row ID': [1],
        'Order ID': ['CA-2016-152156'],
        'Order Date': ['11/8/2016'],
        'Ship Date': ['11/11/2016'],
        'Ship Mode': ['Second Class'],
        'Customer ID': ['CG-12520'],
        'Customer Name': ['Claire Gute'],
        'Segment': ['Consumer'],
        'Country': ['United States'],
        'City': ['Henderson'],
        'State': ['Kentucky'],
        'Postal Code': [42420],
        'Region': ['South'],
        'Product ID': ['FUR-BO-10001798'],
        'Category': ['Furniture'],
        'Sub-Category': ['Bookcases'],
        'Product Name': ['Bush Somerset Collection Bookcase'],
        'Sales': [261.96],
        'Quantity': [2],
        'Discount': [0],
        'Profit': [41.9136]
    }

    df = pd.DataFrame(data)
    print("Created sample dataframe with example data.")

    # Generate more mock data for better analysis
    categories = ['Furniture', 'Office Supplies', 'Technology']
    subcategories = {
        'Furniture': ['Bookcases', 'Chairs', 'Tables', 'Furnishings'],
        'Office Supplies': ['Paper', 'Binders', 'Art', 'Storage'],
        'Technology': ['Phones', 'Machines', 'Accessories', 'Copiers']
    }
    regions = ['South', 'North', 'East', 'West']
    segments = ['Consumer', 'Corporate', 'Home Office']
    ship_modes = ['Standard Class', 'Second Class', 'First Class', 'Same Day']

    import random
    from datetime import timedelta

    # Create base date for order generation
    base_date = datetime.strptime('1/1/2016', '%m/%d/%Y')

    # Generate 500 mock records
    mock_data = []
    for i in range(2, 502):  # Start from 2 since we already have record 1
        category = random.choice(categories)
        subcategory = random.choice(subcategories[category])
        region = random.choice(regions)
        segment = random.choice(segments)
        ship_mode = random.choice(ship_modes)

        # Generate random dates
        order_date = base_date + timedelta(days=random.randint(0, 730))  # Spread over 2 years
        ship_date = order_date + timedelta(days=random.randint(1, 10))  # Ship 1-10 days after order

        # Generate random financial data
        quantity = random.randint(1, 10)
        unit_price = random.uniform(15, 250)
        sales = round(quantity * unit_price, 2)
        discount = round(random.choice([0, 0, 0, 0.1, 0.2, 0.3, 0.4, 0.5]), 2)  # More weight to zero discount
        cost = round(sales * random.uniform(0.4, 0.8), 2)  # Cost is 40-80% of sales
        profit = round(sales - cost, 2)

        if discount > 0:
            sales = round(sales * (1 - discount), 2)
            profit = round(sales - cost, 2)  # Recalculate profit after discount

        record = {
            'Row ID': i,
            'Order ID': f"OR-{random.randint(2016, 2019)}-{random.randint(100000, 999999)}",
            'Order Date': order_date.strftime('%m/%d/%Y'),
            'Ship Date': ship_date.strftime('%m/%d/%Y'),
            'Ship Mode': ship_mode,
            'Customer ID': f"{chr(65 + random.randint(0, 25))}{chr(65 + random.randint(0, 25))}-{random.randint(10000, 99999)}",
            'Customer Name': f"Customer {i}",
            'Segment': segment,
            'Country': 'United States',
            'City': f"City {i % 50}",  # Create 50 different cities
            'State': f"State {i % 15}",  # Create 15 different states
            'Postal Code': random.randint(10000, 99999),
            'Region': region,
            'Product ID': f"PROD-{category[:3]}-{random.randint(10000000, 99999999)}",
            'Category': category,
            'Sub-Category': subcategory,
            'Product Name': f"{subcategory} Product {i % 100}",  # 100 different product names
            'Sales': sales,
            'Quantity': quantity,
            'Discount': discount,
            'Profit': profit
        }
        mock_data.append(record)

    # Add mock data to the dataframe
    df = pd.concat([df, pd.DataFrame(mock_data)], ignore_index=True)
    print(f"Generated a total of {len(df)} records for analysis.")

# Display basic information about the dataset
print("\nDataset Information:")
print(f"Number of records: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("\nColumns in the dataset:")
print(df.columns.tolist())

# Convert date columns to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# Extract year and month for time-based analysis
df['Order Year'] = df['Order Date'].dt.year
df['Order Month'] = df['Order Date'].dt.month
df['Order Day'] = df['Order Date'].dt.day

# Basic statistics of numerical columns
print("\nBasic Statistics of Sales, Quantity, Discount, and Profit:")
print(df[['Sales', 'Quantity', 'Discount', 'Profit']].describe())

# Calculate delivery days (difference between ship date and order date)
df['Delivery Days'] = (df['Ship Date'] - df['Order Date']).dt.days

# Function to create visualizations
def create_visualizations(df):
    # Set the style for the plots
    sns.set(style="whitegrid")

    # 1. Monthly Sales and Profit Trends
    plt.figure(figsize=(14, 8))
    monthly_data = df.groupby(['Order Year', 'Order Month'])[['Sales', 'Profit']].sum().reset_index()
    monthly_data['Date'] = pd.to_datetime(monthly_data['Order Year'].astype(str) + '-' + monthly_data['Order Month'].astype(str) + '-01')
    monthly_data = monthly_data.sort_values('Date')

    plt.subplot(2, 1, 1)
    plt.plot(monthly_data['Date'], monthly_data['Sales'], marker='o', linewidth=2, color='blue')
    plt.title('Monthly Sales Trend', fontsize=14)
    plt.ylabel('Sales ($)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(2, 1, 2)
    plt.plot(monthly_data['Date'], monthly_data['Profit'], marker='o', linewidth=2, color='green')
    plt.title('Monthly Profit Trend', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Profit ($)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('monthly_trends.png')
    plt.close()

    # 2. Category and Sub-Category Analysis
    plt.figure(figsize=(14, 10))

    # Sales by Category
    plt.subplot(2, 2, 1)
    category_sales = df.groupby('Category')['Sales'].sum().sort_values(ascending=False)
    sns.barplot(x=category_sales.index, y=category_sales.values, palette='viridis')
    plt.title('Sales by Category', fontsize=14)
    plt.ylabel('Sales ($)', fontsize=12)
    plt.xticks(rotation=45)

    # Profit by Category
    plt.subplot(2, 2, 2)
    category_profit = df.groupby('Category')['Profit'].sum().sort_values(ascending=False)
    sns.barplot(x=category_profit.index, y=category_profit.values, palette='viridis')
    plt.title('Profit by Category', fontsize=14)
    plt.ylabel('Profit ($)', fontsize=12)
    plt.xticks(rotation=45)

    # Top 10 Sub-categories by Sales
    plt.subplot(2, 2, 3)
    subcategory_sales = df.groupby('Sub-Category')['Sales'].sum().sort_values(ascending=False).head(10)
    sns.barplot(x=subcategory_sales.values, y=subcategory_sales.index, palette='viridis')
    plt.title('Top 10 Sub-Categories by Sales', fontsize=14)
    plt.xlabel('Sales ($)', fontsize=12)

    # Top 10 Sub-categories by Profit
    plt.subplot(2, 2, 4)
    subcategory_profit = df.groupby('Sub-Category')['Profit'].sum().sort_values(ascending=False).head(10)
    sns.barplot(x=subcategory_profit.values, y=subcategory_profit.index, palette='viridis')
    plt.title('Top 10 Sub-Categories by Profit', fontsize=14)
    plt.xlabel('Profit ($)', fontsize=12)

    plt.tight_layout()
    plt.savefig('category_analysis.png')
    plt.close()

    # 3. Regional Performance
    plt.figure(figsize=(14, 10))

    # Sales by Region
    plt.subplot(2, 2, 1)
    region_sales = df.groupby('Region')['Sales'].sum().sort_values(ascending=False)
    sns.barplot(x=region_sales.index, y=region_sales.values, palette='rocket')
    plt.title('Sales by Region', fontsize=14)
    plt.ylabel('Sales ($)', fontsize=12)

    # Profit by Region
    plt.subplot(2, 2, 2)
    region_profit = df.groupby('Region')['Profit'].sum().sort_values(ascending=False)
    sns.barplot(x=region_profit.index, y=region_profit.values, palette='rocket')
    plt.title('Profit by Region', fontsize=14)
    plt.ylabel('Profit ($)', fontsize=12)

    # Top 10 States by Sales
    plt.subplot(2, 2, 3)
    state_sales = df.groupby('State')['Sales'].sum().sort_values(ascending=False).head(10)
    sns.barplot(x=state_sales.values, y=state_sales.index, palette='rocket')
    plt.title('Top 10 States by Sales', fontsize=14)
    plt.xlabel('Sales ($)', fontsize=12)

    # Top 10 States by Profit
    plt.subplot(2, 2, 4)
    state_profit = df.groupby('State')['Profit'].sum().sort_values(ascending=False).head(10)
    sns.barplot(x=state_profit.values, y=state_profit.index, palette='rocket')
    plt.title('Top 10 States by Profit', fontsize=14)
    plt.xlabel('Profit ($)', fontsize=12)

    plt.tight_layout()
    plt.savefig('regional_analysis.png')
    plt.close()

    # 4. Discount Analysis
    plt.figure(figsize=(14, 10))

    # Discount distribution
    plt.subplot(2, 2, 1)
    sns.histplot(df['Discount'], bins=20, kde=True)
    plt.title('Discount Distribution', fontsize=14)
    plt.xlabel('Discount', fontsize=12)

    # Average profit by discount level
    plt.subplot(2, 2, 2)
    df['Discount_Bucket'] = pd.cut(df['Discount'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
    discount_profit = df.groupby('Discount_Bucket')['Profit'].mean().reset_index()
    sns.barplot(x='Discount_Bucket', y='Profit', data=discount_profit, palette='coolwarm')
    plt.title('Average Profit by Discount Level', fontsize=14)
    plt.ylabel('Average Profit ($)', fontsize=12)
    plt.xticks(rotation=45)

    # Scatter plot of Discount vs Profit
    plt.subplot(2, 2, 3)
    sns.scatterplot(data=df, x='Discount', y='Profit', hue='Category', alpha=0.6)
    plt.title('Discount vs. Profit by Category', fontsize=14)
    plt.xlabel('Discount', fontsize=12)
    plt.ylabel('Profit ($)', fontsize=12)

    # Average sales by discount level
    plt.subplot(2, 2, 4)
    discount_sales = df.groupby('Discount_Bucket')['Sales'].mean().reset_index()
    sns.barplot(x='Discount_Bucket', y='Sales', data=discount_sales, palette='coolwarm')
    plt.title('Average Sales by Discount Level', fontsize=14)
    plt.ylabel('Average Sales ($)', fontsize=12)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig('discount_analysis.png')
    plt.close()

    # 5. Ship Mode Analysis
    plt.figure(figsize=(14, 8))

    # Average delivery days by ship mode
    plt.subplot(1, 2, 1)
    ship_delivery = df.groupby('Ship Mode')['Delivery Days'].mean().reset_index()
    sns.barplot(x='Ship Mode', y='Delivery Days', data=ship_delivery, palette='Set2')
    plt.title('Average Delivery Days by Ship Mode', fontsize=14)
    plt.ylabel('Average Delivery Days', fontsize=12)
    plt.xticks(rotation=45)

    # Profit by ship mode
    plt.subplot(1, 2, 2)
    ship_profit = df.groupby('Ship Mode')['Profit'].sum().reset_index()
    sns.barplot(x='Ship Mode', y='Profit', data=ship_profit, palette='Set2')
    plt.title('Total Profit by Ship Mode', fontsize=14)
    plt.ylabel('Profit ($)', fontsize=12)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig('shipmode_analysis.png')
    plt.close()

# Perform in-depth analysis
def perform_analysis(df):
    # 1. Overall Performance Metrics
    total_sales = df['Sales'].sum()
    total_profit = df['Profit'].sum()
    profit_margin = (total_profit / total_sales) * 100
    total_orders = df['Order ID'].nunique()
    avg_order_value = total_sales / total_orders

    print("\nOverall Performance Metrics:")
    print(f"Total Sales: ${total_sales:,.2f}")
    print(f"Total Profit: ${total_profit:,.2f}")
    print(f"Profit Margin: {profit_margin:.2f}%")
    print(f"Total Orders: {total_orders}")
    print(f"Average Order Value: ${avg_order_value:.2f}")

    # 2. Category Analysis
    category_analysis = df.groupby('Category').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique
    }).rename(columns={'Order ID': 'Order Count'})

    category_analysis['Profit Margin (%)'] = (category_analysis['Profit'] / category_analysis['Sales']) * 100
    category_analysis['Avg Order Value'] = category_analysis['Sales'] / category_analysis['Order Count']

    print("\nCategory Performance:")
    print(category_analysis.sort_values('Sales', ascending=False))

    # 3. Subcategory Analysis
    subcategory_analysis = df.groupby(['Category', 'Sub-Category']).agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique
    }).rename(columns={'Order ID': 'Order Count'})

    subcategory_analysis['Profit Margin (%)'] = (subcategory_analysis['Profit'] / subcategory_analysis['Sales']) * 100

    print("\nTop 5 Sub-Categories by Sales:")
    print(subcategory_analysis.sort_values('Sales', ascending=False).head())

    print("\nBottom 5 Sub-Categories by Profit Margin:")
    print(subcategory_analysis.sort_values('Profit Margin (%)').head())

    # 4. Regional Analysis
    regional_analysis = df.groupby('Region').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique
    }).rename(columns={'Order ID': 'Order Count'})

    regional_analysis['Profit Margin (%)'] = (regional_analysis['Profit'] / regional_analysis['Sales']) * 100

    print("\nRegional Performance:")
    print(regional_analysis.sort_values('Sales', ascending=False))

    # 5. Discount Analysis
    df['Discount_Bucket'] = pd.cut(df['Discount'], bins=[0, 0.1, 0.2, 0.3, 0.4, 1],
                                 labels=['0-10%', '10-20%', '20-30%', '30-40%', '40-100%'])

    discount_analysis = df.groupby('Discount_Bucket').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique
    }).rename(columns={'Order ID': 'Order Count'})

    discount_analysis['Profit Margin (%)'] = (discount_analysis['Profit'] / discount_analysis['Sales']) * 100

    print("\nDiscount Level Analysis:")
    print(discount_analysis)

    # 6. Customer Segment Analysis
    segment_analysis = df.groupby('Segment').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Customer ID': pd.Series.nunique,
        'Order ID': pd.Series.nunique
    }).rename(columns={'Customer ID': 'Customer Count', 'Order ID': 'Order Count'})

    segment_analysis['Profit Margin (%)'] = (segment_analysis['Profit'] / segment_analysis['Sales']) * 100
    segment_analysis['Avg Sales per Customer'] = segment_analysis['Sales'] / segment_analysis['Customer Count']

    print("\nCustomer Segment Analysis:")
    print(segment_analysis.sort_values('Sales', ascending=False))

    # 7. Top Customers
    customer_analysis = df.groupby('Customer Name').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique
    }).rename(columns={'Order ID': 'Order Count'})

    customer_analysis['Avg Order Value'] = customer_analysis['Sales'] / customer_analysis['Order Count']

    print("\nTop 10 Customers by Sales:")
    print(customer_analysis.sort_values('Sales', ascending=False).head(10))

    # 8. Ship Mode Analysis
    shipmode_analysis = df.groupby('Ship Mode').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique,
        'Delivery Days': 'mean'
    }).rename(columns={'Order ID': 'Order Count'})

    shipmode_analysis['Profit Margin (%)'] = (shipmode_analysis['Profit'] / shipmode_analysis['Sales']) * 100

    print("\nShip Mode Analysis:")
    print(shipmode_analysis.sort_values('Delivery Days'))

    # 9. Monthly Analysis
    df['YearMonth'] = df['Order Date'].dt.to_period('M')
    monthly_analysis = df.groupby('YearMonth').agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': pd.Series.nunique
    }).rename(columns={'Order ID': 'Order Count'})

    monthly_analysis['Profit Margin (%)'] = (monthly_analysis['Profit'] / monthly_analysis['Sales']) * 100

    print("\nMonthly Performance (Last 6 Months):")
    print(monthly_analysis.sort_index(ascending=False).head(6))

    return {
        'total_sales': total_sales,
        'total_profit': total_profit,
        'profit_margin': profit_margin,
        'category_analysis': category_analysis,
        'subcategory_analysis': subcategory_analysis,
        'regional_analysis': regional_analysis,
        'discount_analysis': discount_analysis,
        'segment_analysis': segment_analysis
    }

# Generate actionable insights based on the analysis
def generate_insights(analysis_results):
    insights = []

    # Overall performance insights
    if analysis_results['profit_margin'] > 15:
        insights.append("The overall profit margin of {:.2f}% is strong, indicating effective pricing strategies.".format(
            analysis_results['profit_margin']))
    else:
        insights.append("The overall profit margin of {:.2f}% could be improved. Consider reviewing pricing or cost structure.".format(
            analysis_results['profit_margin']))

    # Category insights
    category_margins = analysis_results['category_analysis']['Profit Margin (%)']
    best_category = category_margins.idxmax()
    worst_category = category_margins.idxmin()

    insights.append("The {} category has the highest profit margin at {:.2f}%. Consider allocating more resources to this category.".format(
        best_category, category_margins[best_category]))

    if category_margins[worst_category] < 10:
        insights.append("The {} category has a concerning profit margin of {:.2f}%. This category needs immediate attention.".format(
            worst_category, category_margins[worst_category]))

    # Subcategory insights
    subcategory_margins = analysis_results['subcategory_analysis']['Profit Margin (%)']
    negative_margin_subcats = subcategory_margins[subcategory_margins < 0]

    if not negative_margin_subcats.empty:
        insights.append("There are {} sub-categories with negative profit margins. Consider discontinuing or repricing these products.".format(
            len(negative_margin_subcats)))
        for (category, subcategory), margin in negative_margin_subcats.head(3).items():
            insights.append("- {} in {} category has a margin of {:.2f}%".format(subcategory, category, margin))

    # Regional insights
    regional_margins = analysis_results['regional_analysis']['Profit Margin (%)']
    best_region = regional_margins.idxmax()
    worst_region = regional_margins.idxmin()

    insights.append("The {} region is the most profitable with a margin of {:.2f}%. Analyze successful strategies in this region.".format(
        best_region, regional_margins[best_region]))

    if regional_margins[worst_region] < regional_margins[best_region] - 5:
        insights.append("The {} region underperforms with a margin of {:.2f}%. Investigate challenges specific to this region.".format(
            worst_region, regional_margins[worst_region]))

    # Discount insights
    discount_analysis = analysis_results['discount_analysis']
    if '40-100%' in discount_analysis.index:
        high_discount_margin = discount_analysis.loc['40-100%', 'Profit Margin (%)']
        if high_discount_margin < 10:
            insights.append("High discounts (40-100%) result in a low profit margin of {:.2f}%. Review discount policies.".format(
                high_discount_margin))

    # Customer segment insights
    segment_analysis = analysis_results['segment_analysis']
    most_profitable_segment = segment_analysis['Profit Margin (%)'].idxmax()
    highest_value_segment = segment_analysis['Avg Sales per Customer'].idxmax()

    insights.append("The {} segment is the most profitable with a margin of {:.2f}%. Focus marketing efforts on this segment.".format(
        most_profitable_segment, segment_analysis.loc[most_profitable_segment, 'Profit Margin (%)']))

    insights.append("The {} segment has the highest average sales per customer at ${:.2f}. Develop retention strategies for these customers.".format(
        highest_value_segment, segment_analysis.loc[highest_value_segment, 'Avg Sales per Customer']))

    return insights

# Main execution
if __name__ == "__main__":
    try:
        # Basic data cleaning
        # Remove duplicates if any
        df_cleaned = df.drop_duplicates()

        # Handle missing values if any
        df_cleaned = df_cleaned.fillna({
            'Discount': 0,
            'Quantity': 1
        })

        # Create visualizations
        create_visualizations(df_cleaned)

        # Perform in-depth analysis
        analysis_results = perform_analysis(df_cleaned)

        # Generate insights
        insights = generate_insights(analysis_results)

        print("\n===== Key Insights =====")
        for i, insight in enumerate(insights, 1):
            print(f"{i}. {insight}")

        print("\nAnalysis complete! Visualizations saved as PNG files.")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Trying to read file with latin1 encoding...
Successfully read the file with latin1 encoding.

Dataset Information:
Number of records: 9994
Number of columns: 21

Columns in the dataset:
['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

Basic Statistics of Sales, Quantity, Discount, and Profit:
              Sales     Quantity     Discount       Profit
count   9994.000000  9994.000000  9994.000000  9994.000000
mean     229.858001     3.789574     0.156203    28.656896
std      623.245101     2.225110     0.206452   234.260108
min        0.444000     1.000000     0.000000 -6599.978000
25%       17.280000     2.000000     0.000000     1.728750
50%       54.490000     3.000000     0.200000     8.666500
75%      209.940000     5.000000     0.200000    29.364000
max    22638.480000   

<ipython-input-4-d96118097f4d>:185: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=category_sales.index, y=category_sales.values, palette='viridis')
<ipython-input-4-d96118097f4d>:193: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=category_profit.index, y=category_profit.values, palette='viridis')
<ipython-input-4-d96118097f4d>:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=subcategory_sales.values, y=subcategory_sales.index, palette='viridis')
<ipython-input-4-d96118097f4d>:208: FutureWarning: 

Passing `palette` without assigning 


Overall Performance Metrics:
Total Sales: $2,297,200.86
Total Profit: $286,397.02
Profit Margin: 12.47%
Total Orders: 5009
Average Order Value: $458.61

Category Performance:
                       Sales       Profit  Order Count  Profit Margin (%)  \
Category                                                                    
Technology       836154.0330  145454.9481         1544          17.395712   
Furniture        741999.7953   18451.2728         1764           2.486695   
Office Supplies  719047.0320  122490.8008         3742          17.035158   

                 Avg Order Value  
Category                          
Technology            541.550540  
Furniture             420.634805  
Office Supplies       192.155808  

Top 5 Sub-Categories by Sales:
                                   Sales      Profit  Order Count  \
Category        Sub-Category                                        
Technology      Phones        330007.054  44515.7306          814   
Furniture       Chairs  

<ipython-input-4-d96118097f4d>:371: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  discount_analysis = df.groupby('Discount_Bucket').agg({
